In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from packages.rl import env_hhb
from packages.z_sampling.real_xy_sample import generate_gaussian_real_points

In [6]:
def plot_2D_potential_Virasoro(
    central_charge,
    bounds_d1,          # e.g. [d1_min, d1_max]
    bounds_d3,          # e.g. [d3_min, d3_max]
    N_lsq=20,
    kmax=2,
    kmax1=10,
    device="cuda",
    dtype=torch.float64,
    contour_std=0.1
):
    """
    在 (d1, d3) 平面上绘制 Virasoro 最小二乘 reward 的等高线图。  
    说明：
      - 这里我们假设“状态维度 N_state = 1”，也就是 h 和 hb 都是一维标量。  
      - 把 h_ext 固定为 0。若想让 h_ext 也参与变化，可自行扩展。
    """

    # 1) 首先构造 (d1,d3) 网格：
    nx, ny = 200, 200
    xs = np.linspace(bounds_d1[0], bounds_d1[1], nx)
    ys = np.linspace(bounds_d3[0], bounds_d3[1], ny)
    Xg, Yg = np.meshgrid(xs, ys, indexing="xy")  # 形状 (ny, nx) 对应 (d3,d1)

    # 2) 扁平化成 N_points = nx*ny 个点
    pts = torch.from_numpy(
        np.stack([Xg.ravel(), Yg.ravel()], axis=1)   # shape (nx*ny, 2)
    ).to(device=device, dtype=dtype)
    N_points = pts.size(0)  # = nx * ny

    # 3) 从这 N_points 个点中，取出“h”和“hb”：
    d1_vals = pts[:, 0].unsqueeze(1)  # (N_points, 1)
    d3_vals = pts[:, 1].unsqueeze(1)  # (N_points, 1)
    # 在 Virasoro 最小二乘里，h_vals 和 hb_vals 都需要形如 (N_th, N_state)，此处 N_state=1
    h_vals  = d3_vals.clone()         # (N_points, 1)
    hb_vals = d3_vals.clone()         # (N_points, 1)
    # 这里举例把 h = d3, hb = d3；如果你需要 h,d3→h,hb 可以自己更换对应关系

    # 4) 把 h_ext 固定为 0 (也可用其他常数)
    h_ext = torch.zeros((N_points,), device=device, dtype=dtype)  # (N_points,)

    # 5) 准备 Virasoro 采样 z, zbar：用复数采样
    N_z = 400  # 样本点数，必须是 N_lsq 的整数倍
    assert N_z % N_lsq == 0, "N_z must be a multiple of N_lsq"
    # 这里我们用你预先定义的函数产生复数点：经常是 generate_gaussian_complex_points
    z_vals, zbar_vals = generate_gaussian_real_points(N_z, std=contour_std, device=device)
    print(h_vals)
    print(h_ext)
    Rf, _ = env_hhb.least_sq_std_rew_MC_Virasoro(
        h_vals,        # (N_points, 1)
        hb_vals,       # (N_points, 1)
        h_ext,         # (N_points,)
        z_vals,        # (N_z,) complex
        zbar_vals,     # (N_z,) complex
        {"central_charge", torch.tensor(central_charge, device=device)},
        N_lsq=N_lsq,
        kmax=kmax,
        kmax1=kmax1,
        device=device,
        analytic_H=False,
    )

    # 7) 把 reward 拉到 CPU 并 reshape 成 (ny, nx) 以对应 Xg,Yg
    Rg = Rf.detach().cpu().numpy().reshape(ny, nx)

    # 8) 最后绘图
    fig, ax = plt.subplots(figsize=(6, 6))
    cf = ax.contourf(Xg, Yg, Rg, levels=50, cmap="viridis")
    cbar = fig.colorbar(cf, ax=ax)
    cbar.set_label("reward (Vir.$\exp(S)$)")

    ax.set_xlabel("d₁")
    ax.set_ylabel("d₃")
    ax.set_title("Vir.\ Mn. Least‐Sq Reward on (d₁,d₃) Plane")
    plt.tight_layout()
    plt.show()


In [7]:
central_charge = 7/10 - 1e-3
bounds = [(0.02,0.6),(-0.6,-0.01)]
plot_2D_potential_Virasoro(central_charge=central_charge, bounds_d1=bounds[0], bounds_d3=bounds[1])

tensor([[-0.6000],
        [-0.6000],
        [-0.6000],
        ...,
        [-0.0100],
        [-0.0100],
        [-0.0100]], device='cuda:0')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')


AssertionError: 

In [ ]:
N_lsq=20
N_z = 400
kmax=2
kmax1=10
device="cuda"
dtype=torch.float64
contour_std=0.1

z_vals, zbar_vals = generate_gaussian_real_points(N_z, std=contour_std, device=device)

h_vals = torch.tensor([[3/10]], device=device, dtype=torch.float64).unsqueeze(0)
hb_vals = torch.tensor([[3/10]], device=device).unsqueeze(0)
h_ext = torch.tensor([7/10], device=device).unsqueeze(0)
Rf, _ = env_hhb.least_sq_std_rew_MC_Virasoro(
    h_vals,        # (N_points, 1)
    hb_vals,       # (N_points, 1)
    h_ext,         # (N_points,)
    z_vals,        # (N_z,) complex
    zbar_vals,     # (N_z,) complex
    torch.tensor([[central_charge]], device=device),
    N_lsq=N_lsq,
    kmax=kmax,
    kmax1=kmax1,
    device=device,
    analytic_H=False,
)

ValueError: too many values to unpack (expected 2)

In [ ]:
virasoro_info = {
    "central_charge": 3/2,
    "V_c": torch.tensor([0], device="cuda", dtype=torch.float64),
    "V_d": torch.tensor([0], device="cuda", dtype=torch.float64),
}

# h_vals, hb_vals: torch.Tensor on GPU, dtype=torch.float64
#   形状 (Nth, p) —— Nth 是把所有 trial spectrum 的数目
h_vals = torch.tensor([[3/10]], device=device, dtype=torch.float64)#torch.randn((Nth, p), device="cuda", dtype=torch.float64)
hb_vals = torch.tensor([[3/10]], device=device, dtype=torch.float64)#torch.randn((Nth, p), device="cuda", dtype=torch.float64)

# h_ext: torch.Tensor on GPU, dtype=torch.float64，长度 Nth
h_ext = torch.tensor([7/10], device=device, dtype=torch.float64)#torch.randn((Nth,), device="cuda", dtype=torch.float64)

z_vals, zbar_vals = generate_gaussian_real_points(N_z, std=contour_std, device=device)

# 现在就可以调用
r, c_mean, c_std = env_hhb.least_sq_std_rew_Virasoro(
    h_vals,       # (Nth, p)
    hb_vals,      # (Nth, p)
    h_ext,        # (Nth,)
    z_vals,       # (N_z,)
    zbar_vals,       # (N_z,)
    virasoro_info,
    N_lsq=20,
    kmax=3,
    n_states_rew=2,
    kmax1=10,
    device="cuda",
    analytic_H=False
)

AssertionError: NN must be even

In [ ]:
from CFT_spectrum.Get_spec_to_store import from_json_2_RL_input, load_minimal_pq, get_non_identity_global_p, load_fusion_rule, process_data, process_data_sep

num_states=25
picked_theory = process_data(num_states, [[5, 3], -0.05], output_num_states=None)#from_json_2_RL_input(load_minimal_pq(label="all")[model_idx], num_states=num_states, label='all')
    
delta = torch.tensor(picked_theory["init_state"], device=device)
spin = torch.tensor(picked_theory["spins"], device=device)
cs = np.array(picked_theory["cs"])
dSigma = picked_theory["dSigma"]
central_charge = picked_theory["central charge"]
d_max = int(max(delta))#picked_theory["d_max"]
print(d_max)
print(picked_theory["d_max"])
spec = [delta, spin, cs, dSigma, d_max]
print(f"\nTheory info: \n    theory: {picked_theory['theory']}\n    central charge: {central_charge}\n    d max: {picked_theory['d_max']}\n    dSigam: {dSigma}\n    Delta: {spec[0].tolist()}\n    spin:  {spec[1].tolist()}\n    cs: {cs}\n")

spec, cs = get_non_identity_global_p(spec, num_states=9)
print(f"\nTheory info: \n    theory: {picked_theory['theory']}\n    central charge: {central_charge}\n    d max: {spec[3]}\n    dSigam: {dSigma}\n    Delta: {spec[0].tolist()}\n    spin:  {spec[1].tolist()}\n    cs: {cs}\n")


24
24.4

Theory info: 
    theory: (p, q)=[5, 3]
    central charge: -0.6
    d max: 24.4
    dSigam: -0.1
    Delta: [2.0, 0.4, 2.4, 4.0, 4.0, 4.4, 4.4, 6.0, 6.0, 6.4, 6.4, 8.0, 8.0, 8.0, 8.4, 8.4, 8.4, 10.0, 10.0, 10.4, 10.4, 10.4, 12.0, 12.0, 12.4, 12.4, 12.4, 12.4, 14.0, 14.4, 14.4, 14.4, 16.0, 16.4, 16.4, 16.4, 18.4, 18.4, 20.4, 20.4, 22.4, 24.4]
    spin:  [2.0, 0.0, 2.0, 4.0, 0.0, 0.0, 4.0, 6.0, 2.0, 2.0, 6.0, 8.0, 0.0, 4.0, 4.0, 0.0, 8.0, 2.0, 6.0, 6.0, 2.0, 10.0, 0.0, 4.0, 8.0, 4.0, 0.0, 12.0, 2.0, 10.0, 6.0, 2.0, 0.0, 8.0, 4.0, 0.0, 6.0, 2.0, 4.0, 0.0, 2.0, 0.0]
    cs: [-8.33333333e-03 -1.36869447e-01  3.99943190e-03 -4.93421053e-05
  3.47222222e-05 -2.92166291e-05  3.73585278e-05 -7.29868853e-07
  4.11184211e-07 -5.45822647e-07  8.08021489e-07 -1.21090324e-08
  1.21732168e-09  6.08224044e-09 -1.18055088e-08 -2.54925337e-09
  2.34646066e-08  3.60132658e-11  1.00908603e-10 -3.42827044e-10
 -1.10274769e-10  7.93053711e-10  2.66354271e-13  5.97485150e-13
 -1.15868237e-11 -3.202

In [ ]:
x_vals, y_vals = generate_gaussian_real_points(N_z, std=contour_std, device=device)

env_hhb.least_sq_std_rew_G(spec[0].unsqueeze(0), spec[1].unsqueeze(0), x_vals, y_vals, dSigma, N_lsq=32)

AttributeError: 'float' object has no attribute 'is_cuda'

In [ ]:
spec

[tensor([0.4000, 2.4000, 4.4000, 4.4000, 6.4000, 6.4000, 8.4000, 8.4000, 8.4000],
        device='cuda:0'),
 tensor([0., 2., 0., 4., 2., 6., 4., 0., 8.], device='cuda:0'),
 -0.1,
 8]